In [1]:
#from https://www.jitsejan.com/using-scrapy-in-jupyter-notebook.html
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('linksResult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "links"
    start_urls = [ 'https://en.wikipedia.org/wiki/Philosophy' ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'linksResult.json'                        # Used for pipeline 2
    }
    
    #Todo: fix parsing for wikipedia
    def parse(self, response):
        for link in response.css('div.mw-parser-output a'):
            linkString = str(link.css('::attr(href)').extract_first())
            if ( (linkString[0:6] == '/wiki/') & (linkString[6:11] != 'File:') ):
                yield {
                    'text': linkString,
                }

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2020-12-16 11:22:30 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 11:22:30 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Jul 28 2020, 12:59:40) - [GCC 9.3.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1f  31 Mar 2020), cryptography 2.8, Platform Linux-5.4.0-58-generic-x86_64-with-glibc2.29
2020-12-16 11:22:30 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-16 11:22:30 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2020-12-16 11:22:30 [py.warnings] WARNING: /home/jonathan/.local/lib/python3.8/site-packages/scrapy/extensions/feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)



In [5]:
import pandas as pd
dfjl = pd.read_json('linksResult.jl', lines=True)
dfjl

,text
0,/wiki/Philosophy_(disambiguation)
1,/wiki/The_School_of_Athens
2,/wiki/Raphael
3,/wiki/Ancient_Greek_philosophy
4,/wiki/Ancient_Greek_architecture
...,...
1917,/wiki/LCCN_(identifier)
1918,/wiki/LNB_(identifier)
1919,/wiki/NARA_(identifier)
1920,/wiki/NDL_(identifier)
